In [2]:
import pandas as pd
import numpy as np
import os
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = None
pd.set_option("display.float_format", lambda x: '%.2f' % x)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
os.getcwd()

'/Users/macbook/Documents/workspace_datahackerman/datahakerman_final_project'

In [4]:
# Load the merged dataset for exploration

In [5]:
love_island_data = pd.read_csv('outputs/all_merged_data.csv')
print("'all_merged_data' successfully loaded...")

'all_merged_data' successfully loaded...


In [6]:
love_island_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      3000 non-null   int64  
 1   author.properties.friends       3000 non-null   object 
 2   author.properties.status_count  2999 non-null   float64
 3   author.properties.verified      3000 non-null   object 
 4   content.body                    2999 non-null   object 
 5   location.country                2999 non-null   object 
 6   properties.platform             2998 non-null   object 
 7   properties.sentiment            2999 non-null   float64
 8   location.latitude               2999 non-null   float64
 9   location.longitude              2999 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 234.5+ KB


### Checking for null values

In [33]:
# Function to check for missing data
def check_null(x):
    # Counts missing data values
    missing_data_count=x.isnull().sum().sort_values(ascending = False)
    percentage = (x.isnull().sum()/x.isnull().count()*100).sort_values(ascending = False)
    # Missing data statistics, if any
    missing_data = pd.concat([missing_data_count,percentage],axis=1,keys=['Missing Data','Percentage'])
    # Note the axis values for row and column respectively
    # Checks how many rows with missing data
    missing_data_rows=pd.isnull(x).any(axis=1).sum()
    # Checks how many columns with missing data
    missing_data_columns=pd.isnull(x).any(axis=0).sum()
    # Print summary information
    print ("The checked data has" + " " + str(x.shape[1]) + " " + " " + "columns.\n" \
           "There are" + " " + str(missing_data_columns) + " " + \
           "columns with missing data and...\n" + \
           "...a total of" + " " + str(missing_data_rows)+ " " + \
           "rows with missing data.")
    x_null = x[x.isnull().any(axis=1)]
    print (missing_data)
    # Return missing data
    return x_null

In [34]:
check_null(love_island_data)

The checked data has 10  columns.
There are 7 columns with missing data and...
...a total of 2 rows with missing data.
                                Missing Data  Percentage
properties.platform                        2        0.07
author.properties.status_count             1        0.03
content.body                               1        0.03
location.country                           1        0.03
properties.sentiment                       1        0.03
location.latitude                          1        0.03
location.longitude                         1        0.03
Unnamed: 0                                 0        0.00
author.properties.friends                  0        0.00
author.properties.verified                 0        0.00


,Unnamed: 0,author.properties.friends,author.properties.status_count,author.properties.verified,content.body,location.country,properties.platform,properties.sentiment,location.latitude,location.longitude
1551,551,854,3688.00,False,@DrunkenOldQrow @FancyWeiss,GB,NaN,1.00,52.05,-2.70
1552,552,|| TELL ME YOUR NAME! XD,NaN,twitter,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
love_island_data1552 = love_island_data.iloc[1551,:]

In [37]:
love_island_data["properties.platform"].value_counts()

properties.platform
twitter    2998
Name: count, dtype: int64

In [36]:
love_island_data1552 

Unnamed: 0                                                 551
author.properties.friends                                  854
author.properties.status_count                         3688.00
author.properties.verified                               False
content.body                      @DrunkenOldQrow @FancyWeiss 
location.country                                            GB
properties.platform                                        NaN
properties.sentiment                                      1.00
location.latitude                                        52.05
location.longitude                                       -2.70
Name: 1551, dtype: object

In [38]:
check_null(love_island_data)

The checked data has 10  columns.
There are 7 columns with missing data and...
...a total of 2 rows with missing data.
                                Missing Data  Percentage
properties.platform                        2        0.07
author.properties.status_count             1        0.03
content.body                               1        0.03
location.country                           1        0.03
properties.sentiment                       1        0.03
location.latitude                          1        0.03
location.longitude                         1        0.03
Unnamed: 0                                 0        0.00
author.properties.friends                  0        0.00
author.properties.verified                 0        0.00


,Unnamed: 0,author.properties.friends,author.properties.status_count,author.properties.verified,content.body,location.country,properties.platform,properties.sentiment,location.latitude,location.longitude
1551,551,854,3688.00,False,@DrunkenOldQrow @FancyWeiss,GB,NaN,1.00,52.05,-2.70
1552,552,|| TELL ME YOUR NAME! XD,NaN,twitter,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# source: https://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn

from sklearn.processing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean', filler = 'NA'):
        self.strategy = strategy
        self.filler = filler
        
    def fit(self, X, y=None):
        if self.strategy in ['mean','median']:
            if not all(X.dtypes == np.number):
                raise ValueError('dtypes mismatch np.number dtype is required for' + self.strategy)
            if self.strategy == 'mean':
                self.fill = X.mean()
        elif self.strategy == 'median':
            self.fill - X.median()
        elif self.strategy == 'mode':
            self.fill - X.mode(),iloc[0]
        elif self.strategy == 'median':
            self.fill - X.median()
        elif self.strategy == 'fill':
            if type(self.fill) is list and type(X) is pd.DataFrame:
                self.fill = ([(cname,v) for cname,v in zip(X.columns, self.fill)])
        return self
    
    def transform(self, X, y=None):
        return X.fillna(self.fill)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# adjusting the data types

In [ ]:
type(love_island_data['author.properties.friends'])

In [ ]:
love_island_data['author.properties.friends'] = pd.to_numeric(love_island_data['author.properties.friends'])

In [ ]:
love_island_data

In [ ]:
all_merged_data['author.properties.friends'] = all_merged_data['author.properties.friends'].astype(int64)

In [ ]:
all_merged_data['location.country'].unique()

## Table 1 - Country Statistics

In [ ]:
data_country = all_merged_data['location.country'].value_counts()
data_country

Country Code | Status Count| Countries |
| --- | --- | --- |
| GB | 2991 | Great Britain |
| GG | 5    | Guernsey |
| JE | 2    | Jersey |
| IM | 1    | Isle of Man |

## Plot 1 - Sentiment Counts

In [ ]:
data_sentiment = all_merged_data['properties.sentiment'].value_counts()
data_sentiment

In [ ]:
import matplotlib.pyplot as plt
sent_count = np.array([1403, 968, 628])
sentiment = ["Negative", "Neutral", "Positive"]
sent_colors = ["blue", "yellow", "red"]

plt.pie(sent_count, labels = sentiment, colors = sent_colors)
print(f"The sentiments are positive 1, neutral 0 and negative -1")
plt.show()

## Plot 2 - Distribution of Friends and Status Count Relationship

In [ ]:
merged_data_copy = all_merged_data.copy()
merged_data_copy['LogStatusCount'] = np.log(merged_data_copy['author.properties.status_count'])
merged_data_copy['LogFriends'] = np.log(merged_data_copy['author.properties.friends'])

In [ ]:
merged_data_copy['LogStatusCount'].min(), merged_data_copy['LogStatusCount'].max()

In [ ]:
merged_data_copy['LogFriends'].min(), merged_data_copy['LogFriends'].max()

In [ ]:
all_merged_data['author.properties.friends'].corr(all_merged_data['author.properties.status_count'])